<a href="https://colab.research.google.com/github/Junoflows/DeepLearning_From_Scratch1/blob/main/Chapter_5_%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5 오차역전파법

+ 가중치 매개변수의 기울기를 효울적으로 계산하는 방법이다.
+ 오차역전파법을 계산 그래프를 통해 시각적으로 이해해보자.

## 5.1 계산 그래프

+ 계산 그래프는 계산 과정을 그래프로 나타낸 것이다.
+ 그래프는 복수의 노드와 에지로 표현된다. (노드 사이의 직선을 에지라고 한다)


### 5.1.1 계산 그래프로 풀다.

+ 간단한 문제를 계산 그래프를 사용해서 풀어보자.

__문제1__  
현빈이가 슈퍼에서서 1개에 100원인 사과를 2개 샀다.  
이때 지불 금액을 구하여라. 단, 소비세가 10% 부과된다.

+ 계산 그래프는 계산 과정을 노드와 화살표로 표현한다.
+ 노드는 원으로 표기하고 원 안에 연산 내용을 적는다.

<img src='http://drive.google.com/uc?export=view&id=1X0DjjN6XFBNmQ-Zt9PnjGfmnAO1HXVfJ' width = 550 /><br>

+ ×2와 ×1.1 을 각각 하나의 연산으로 취급하지말고 곱셈인 ×만 연산으로 생각해보자.
+ '2'와 '1.1'은 각각 '사과의 개수'와 '소비세' 변수가 되어 원 밖에 표시된다.

<img src='http://drive.google.com/uc?export=view&id=1isLxoyt3mJxaUuU4fUQnEWwdNrlQIne1' width = 550><br>

__문제2__  
현빈이가 슈퍼에서 사과 2개, 귤 3개를 샀다.  
사과는 1개에 100원, 귤 1개에 150원이다. 소비사 10%일 때 지불 금액을 구하시오.

+ 문제2도 계산 그래프로 풀어보자.

<img src='http://drive.google.com/uc?export=view&id=1g_2xEcL5Mf6DVAYDubqwrFLjb9a26VQc' width = 550 /><br>

+ 이 문제에는 덧셈 노드가 새로 등장하여 사과와 귤의 금액을 합산한다.
+ 왼쪽에서 오른쪽으로 계산을 진행하면 답은 715원이다.

+ 계산 그래프를 이용한 문제풀이는 다음 흐름으로 진행한다.

1. 계산 그래프를 구성한다.
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.

+ 여기서 2번째 '계산을 왼쪽에서 오른쪽으로 진행' 하는 단계를 __순전파__라고 한다.
+ 순전파의 반대 방향의 전파도 가능한데 그것을 __역전파__ 라고 한다.
+ 역전파는 이후에 미분을 계산할 때 중요한 역할을 한다.

### 5.1.2 국소적 계산

+ 계산 그래프는 국소적 계산을 전파함으로써 최종 결과를 얻는 특징이 있다.
+ 국소적이란 '자신과 직접 관계된 작은 범위' 라는 뜻이다.
+ 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있다.
+ 국소적 계산의 예를 살펴보자.

<img src='http://drive.google.com/uc?export=view&id=1PzbkfRY2iA_K9VBdjzwaLSNxWyxNbAOT' width = 550><br>

+ 위 예에서는 여러 식품을 구입하여 (복잡한 계산을 거쳐) 총 금액이 40,000원이 되었다.
+ 핵심은 각 노드에서의 계산은 국소적 계산이라는 점이다.
+ 사과와 그 외의 물품 값을 더하는 계산(4,000+200 → 4,200)은  
4,000 이라는 숫자가 어떻게 계산되었는지 상관없이 단지 두 숫자를 더하면 된다는 뜻이다.
+ 각 노드는 자신과 관련한 계산 외에는 아무것도 신경 쓸 게 없다.
+ 국소적인 계산은 단순하지만 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 할 수 있다.


### 5.1.3 왜 계산 그래프로 푸는가?

계산 그래프의 이점

+ 전체가 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화할 수 있다.
+ 중간 계산 결과를 모두 보관할 수 있다.
+ 역전파를 통해 '미분'을 효율적으로 계산할 수 있다.

계산 그래프에서의 역전파

+ 문제 1은 사과를 2개사서 소비세를 포함한 최종 금액을 구하는 것이다.
+ 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지 알고 싶다고 하자.  
 이는 사과 가격에 대한 지불 금액의 미분을 구하는 문제이다.
+ 사과 값을 $x$, 지불 금액을 $L$이라 했을 때 $ \frac{∂L}{∂x}$ 을 구하는 것이다.
+ 아래 그림처럼 계산 그래프 상의 역전파에 의해 미분을 구할 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1ulSa6QvmojR2snm1HVgWMqBykjkJ1kVl' width = 550><br>

+ 역전파는 순전파와는 반대 방향의 화살표(굵은 선)로 그린다.
+  이 전파는 '국소적 미분'을 전달하고 미분 값은 화살표 아래에 적는다.
+ 이 예에서 역전파는 오른쪽에서 왼쪽으로 1→1.1→2.2 순으로 미분 값을 전달한다.

+ 위 예에서는 사과 가격에 대한 미분만 구했지만 다른 변수에 대한 미분도 같은 순서로 구할 수 있다.
+ 또한 중간까지 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산할 수 있다.
+ 계산 그래프는 순전파와 역전파를 활용하여 각 변수의 미분을 효율적으로 구할 수 있다.

## 5.2 연쇄법칙

+ 순전파는 계산 결과를 왼쪽에서 오른쪽으로 전달하지만  
역전파는 국소적인 미분을 오른쪽에서 왼쪽으로 전달한다.
+ 역전파의 원리는 __연쇄법칙__에 따른 것이다.
+ 연쇄법칙이 계산 그래프 상의 역전파와 같다는 것을 보이자.

### 5.2.1 계산 그래프의 역전파

+ $ y = f(x) $ 라는 계산의 역전파를 계산 그래프로 그려보자.

<img src='http://drive.google.com/uc?export=view&id=18OCPoVccKOTVKF5hn51OCIYFCXfEUm9l' width = 550/><br>

+ 역전파의 계산 절차는 신호 $E$에 노드의 국소적 미분( $\frac{∂y}{∂x}$)을 곱한 후 다음 노드로 전달하는 것이다.
+ 국소적 미분이란 순전파 때의 $ y = f(x)$ 계산의 미분을 구한다는 것이며,  
이는 x에 대한 y의 미분 ( $\frac{∂y}{∂x}$)을 구한다는 뜻이다.
+ 이 국소적인 미분을 상류에서 전달된 값에 곱해 앞쪽 노드로 전달하는 것이다.

### 5.2.2 연쇄법칙이란?

+ 연쇄법칙은 합성 함수의 미분에 대한 성질이다.
+ 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.
+ 수식으로는 다음과 같다.

$ \Large \frac{∂z}{∂x} = \frac{∂z}{∂t}\frac{∂t}{∂x}$

+ 간단한 예를 살펴보자.

$ z = t^2 \\ t = x + y$

연쇄법칙을 써서 미분 $ \frac{∂z}{∂x}$을 구해보자.

$\Large \frac{∂z}{∂t} = 2t ,  \frac{∂t}{∂x} = 1$ <br/><br/>
$\Large \frac{∂z}{∂x} = \frac{∂z}{∂t}\frac{∂t}{∂x} = 2t·1 = 2(x+y) $

### 5.2.3 연쇄법칙과 계산 그래프

+ 연쇄법칙 계산을 계산 그래프로 나태내보자.

<img src='http://drive.google.com/uc?export=view&id=1V9r6z4Ws46rpgJ4SHbLzguigm_afu9Qu' width = 550><br>

+ 노드로 들어온 입력 신호에 그 노드의 국소적 미분(편미분)을 곱한 후 다음 노드로 전달한다.
+ 위 예를 보면 입력은 $\frac{∂z}{∂z}$ 이고 편미분인 $ \frac{∂z}{∂t}$을 곱하고 다음 노드로 넘긴다.
+ 맨 왼쪽 역전파를 주목하자.  
$ \frac{∂z}{∂z}\frac{∂z}{∂t}\frac{∂t}{∂x} =\frac{∂z}{∂t}\frac{∂t}{∂x} = \frac{∂z}{∂x} $ 가 성립되어 'x에 대한 z의 미분'이 된다. <br/>

+ 즉 역전파가 하는 일은 연쇄법칙의 원리와 같다.

## 5.3 역전파

+ '+'와 '×' 등의 연산을 예로 역전파의 구조를 설명한다.

### 5.3.1 덧셈 노드의 역전파

$ z = x + y $  을 대상으로 역전파를 살펴보자. 우선 미분은 다음과 같이 계산된다.


$ \Large \frac{∂z}{∂x} = 1 $  

$ \Large \frac{∂z}{∂y} = 1 $

이를 계산 그래프로 다음과 같이 그릴 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1H5hH4IsMO6jS24MJu8FI9PNUSaIZX5LL' width = 550><br>

+ 덧셈 노드의 역전파는 1을 곱하기만 할 뿐 입력된 값을 그대로 다음 노드로 보낸다.  
+ 이 예에서는 상류에서 전해진 미분 값을  $\frac{∂L}{∂z}$이라 했는데,  
이는 최종적으로 L 값을 출력하는 큰 계산그래프를 가정하기 때문이다.
+ $ z = x+y$ 계산은 상류로부터 $\frac{∂L}{∂z}$ 값이 전해지고, 하류로는 $\frac{∂L}{∂x}$과 $\frac{∂L}{∂y}$ 값을 전달한다.

<img src='http://drive.google.com/uc?export=view&id=1vGm0-UIkLoVWzWRSsCcyWXmfoU_Uogcw' width = 550 /><br>

+ 구체적인 예를 살펴보면, '$10 + 5 = 15$' 라는 계산이 있고, 상류에서 1.3 값이 흘러온다. 계산 그래프로 그리면 다음과 같다.
+ 덧셈 노드 역전파는 입력 신호를 다음 노트로 출력할 뿐이고, 1.3을 그대로 다음 노드로 전달한다.

<img src='http://drive.google.com/uc?export=view&id=1ZepeHZn_pPjkXrBa9eR3Sv6P0yNMdJLN' width = 550><br>

### 5.3.2 곱셈 노드의 역전파

+ $z = xy$ 라는 식을 보자. 이 식의 미분은 다음과 같다.

$ \Large \frac{∂z}{∂x} = y $  

$ \Large \frac{∂z}{∂y} = x $

계산 그래프는 다음과 같이 그릴 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1go7BW4Y4YBJrYXNaGGYyIrxMk0hjLlF3' width = 550><br>

+ 곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 보낸다.
+ 서로 바꾼 값이란 위 그림처럼 순전파 때의 $x$ 였다면 역전파에서는 $y$,  
순전파 때 $y$ 였다면 역전파에서는 $x$로 바꾼다는 의미이다.

+ 구체적인 예를 살펴보면, '$10 × 5 = 50$' 이라는 계산이 있고, 역전파 때 상류에서 1.3 값이 흘러온다고 하자.
+ 이를 계산 그래프로 그리면 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=127IneGmdQUlQKGfU7b6dgkvAhwVmIz25' width = 550 /><br>

+ 입력 신호를 바꾼 값을 곱하여 하나는 $1.3 × 5 = 6.5$ , 다른 하나는 $1.3 × 10 = 13$ 이 된다.
+ 덧셈의 역전파와 달리, 곱셈의 역전파는 순방향 입력 신호의 값이 필요하다.  
그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둔다.

### 5.3.3 사과 쇼핑의 예

+ 사과 쇼핑 예를 다시 살펴보자.
+ 사과의 가격, 사과의 개수, 소비세라는 세 변수 각각이 최종 금액에 어떻게 영향을 주느냐를 풀고자 한다.
+ '사과 가격에 대한 지불 금액의 미분', '사과 개수에 대한 지불 금액의 미분',  
'소비세에 대한 지불 금액의 미분'을 구하는 것이 해당된다.
+ 이를 계산 그래프의 역전파를 사용해서 풀면 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=16sDIRSasisHs6SaxK97vN9RPF1rCsc9A' width = 550/><br>

+ 사과 가격의 미분은 2.2, 사과 개수의 미분은 110, 소비세의 미분은 200이다.
+ 이는 소비세와 사과 가격이 같은 양만큼 오르면 최종 금액에는 소비세가 200의 크기로,  
사과 가격이 2.2 크기로 영향을 준다고 해석할 수 있다.
+ 단, 이 예에서 소비세와 사과 가격은 단위가 다르니 주의해야 한다.(소비세 1은 100%, 사과 가격 1은 1원).
+ 마지막으로 '사과와 귤 쇼핑'의 역전파를 풀어보자. 빈칸에 각 변수의 미분을 구해서 채워보자.  
(정답은 뒤에 나옴)

<img src='http://drive.google.com/uc?export=view&id=1gdjwvahGj0aYmto0yVc5IOQFnp9FC3wL' width = 550 /><br>

## 5.4 단순한 계층 구현하기

+ 앞서 보아온 '사과 쇼핑' 예를 파이썬으로 구현해보자.
+ 계산 그래프의 곱셈 노드를 'MulLayer', 덧셈 노드를 'AddLayer'라는 이름으로 구현한다.

### 5.4.1 곱셈 계층

+ 모든 계층은 forward( )와 backward( )라는 공통의 매서드를 갖도록 구현한다.
+ forward( )는 순전파, backward( )는 역전파를 처리한다.

+ 곱셈 계층은 다음과 같이 구현한다.

In [ ]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y

    return out

  def backward(self, dout):
    dx = dout * self.y # x와 y를 바꾼다.
    dy = dout * self.x

    return dx, dy

+ _ _ init _ _( )에서 인스턴스 변수인 x, y를 초기화한다. 두 변수는 순전파 시의 입력 값을 유지하기 위해 사용한다.
+ forward( )에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다.
+ backward( )에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 '서로 바꿔' 곱한 후 하류로 흘린다.

+ 이 MulLayer을 사용해서 앞서 본 '사과 쇼핑'을 구현해보자.

<img src='http://drive.google.com/uc?export=view&id=1Jrbk5vQIcNDWQdmIaS_ehH5FEIDup9ng' width = 550 /><br>

In [ ]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


+ 각 변수에 대한 미분은 backward( )에서 구할 수 있다.

In [ ]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


+ backward( ) 호출 순서는 forward( )때와는 반대이고, backward( )가 받는 인수는 '순전파의 출력에 대한 미분'이다.
+ mul_apple_layer 라는 곱셈 계층은 순전파 때는 apple_price를 출력하지만,  
역전파 때는 apple_price 의 미분 값인 dapple_price를 인수로 받는다.

### 5.4.2 덧셈 계층

+ 덧셈 계층을 구현해보자.

In [ ]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

+ 덧셈 계층에서는 초기화가 필요없으니 _ _ init _ _( )에서는 아무 일도 하지 않는다.
+ forward( )에서는 입력받은 두 인수 x, y를 더해서 반환한다.
+ backward( )에서는 상류에서 내려온 미분(dout)을 그대로 하류로 흘린다.

+ 덧셈 계층과 곱셈 계층을 사용하여 사과 2개와 귤 3개를 사는 상황을 구현해보자.

<img src='http://drive.google.com/uc?export=view&id=1HDDb9fyPvCF0vEQSHMu7KOzei2pXfBAe' width = 550/><br>

In [ ]:
apple = 100
orange = 150
apple_num = 2
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()



# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, apple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple, dapple_num, dorange, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


+ 필요한 계층을 만들어 순전파 메서드인 forward( )를 적절한 순서로 호출한다.
+ 순전파와 반대 순서로 역전파 메서드인 backward( )를 호출하면 원하는 미분이 나온다.

## 5.5 활성화 함수 계층 구현하기

+ 계산 그래프를 신경망에 적용해보자.
+ 신경망을 구성하는 층 각각을 클래스 하나로 구현한다.
+ 활성화 함수인 ReLU와 Sigmoid 계층을 구현한다.

### 5.5.1 ReLU 계층

+ 활성화 함수로 사용되는 ReLU의 수식은 다음과 같다.

$y =
\begin{cases}
x \;(x > 0)\\
0 \;(x \leq 0)
\end{cases}$

+ x에 대한 y의 미분은 다음과 같다.

$ \frac{∂y}{∂x} =
\begin{cases}
1 \;(x > 0)\\
0 \;(x \leq 0)
\end{cases}$

+ 순전파 때의 입력인 $x$가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘린다.
+ 순전파 때 $x$가 0 이하면 역전파 때는 하류로 신호를 보내지 않는다.
+ 계산 그래프로 표현은 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=1sj0LrwDwrX2oZ4CyODUpUEe8Y-CNkAbS' width = 550/><br>

+ 신경망 계층의 forward( )와 backward( ) 함수는 넘파이 배열을 인수로 받는다고 가정하고 ReLU 계층을 구현해보자.

In [ ]:
class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout

    return dx

+ Relu 클래스는 mask라는 인스턴스 변수를 가진다.
+ mask는 True/False로 구성된 넘퍼아 배열로,  
순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외는 False로 유지한다.

In [ ]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]] )
print(x)

mask = (x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


+ 순전파 때의 입력 값이 0 이하면 역전파 떄의 값은 0이 돼야 한다.
+ 그래서 역전파 때는 순전파 때 만들어둔 mask를 써서 mask 원소가 True 인 곳에는 상류에서 전파된 dout을 0으로 설정한다.

### 5.5.2 Sigmoid 계층

+ 시그모이드 함수는 다음 식을 의미하는 함수이다.

$ \Large y = \frac{1}{1+ e^{-x}}$

+ 계산 그래프로 그리면 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=1Xh6KGjX6ncsV2A_CV15dTC6qNBPrToAV' width = 550/><br>

+ 'exp'와 '/' 노드가 새롭게 등장했다.  
'exp'노드는 y = exp(x) 계산을 수행하고 '/' 노드는 y = $\frac{1}{x}$ 계산을 수행한다.
+ 위 식의 계산은 국소적 계산의 전파로 이뤄진다.
+ 위 식의 역전파를 알아보자.

__1단계__
+ $ y = \frac{1}{x}$을 미분하면 다음 식이 된다.  

+ $ \Large \frac{\partial y}{∂x} = -\frac{1}{x^{2}} = - y^{2}$

+ 역전파 때는 상류에서 흘러온 값에 $-y^{2}$ 을 곱해서 하류로 전달한다.
+ 계산 그래프에서는 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=1A3CbZ9cngWh0rqY58qHlhOzWfZ9H6UgP' width = 550/><br>

__2단계__
+ '+' 노드는 상류의 값을 여과 없이 하류로 내보낸다. 계산 그래프는 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=1ojRDt_ueWc6VoWMX8lOVnoUdjdt0Ik6l' width = 550/><br>

__3단계__
+ 'exp' 노드는 $y = e^{x}$ 연산을 수행하며, 미분은 다음과 같다.

+  $ \Large \frac{\partial y}{∂x} = e^{x}$

+ 계산 그래프에서는 상류의 값에 순전파 때의 출력을 곱해 하류로 전파한다.

<img src='http://drive.google.com/uc?export=view&id=1Bq73iuvMFYxAugSL9cuB44uZ0lu5iToQ' width = 550/><br>

__4단계__
+ '×' 노드는 순전파 때의 값을 '서로 바꿔' 곱한다. 이 예에서는 -1을 곱하면 된다.

<img src='http://drive.google.com/uc?export=view&id=1ksAFP0GEUDMfJTLfFMtyy4ytOfipQkNs' width = 550 /><br>

+ 역전파의 최종 출력인 $\frac{∂L}{∂y} y^{2}e^{-x}$ 값이 하류 노드로 전파된다.
+ 순전파의 입력 $x$와 출력 $y$만으로  $\frac{∂L}{∂y} y^{2}e^{-x}$ 를 계산할 수 있다.
+ 계산 그래프의 중간 과정을 모두 묶어서 아래처럼 단순한 'sigmoid' 노드 하나로 대체할 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1xpbcWlL4LN_wxln-oh4TwXZ0HANUH_Oi' width = 550 /><br>

+ 간소화 버전은 역전파 과정의 중간 계산들을 생략할 수 있어 효율적으로 계산할 수 있다.
+ 노드를 그룹화하여 Sigmoid 계층의 세세한 내용을 노출하지 않고 입력과 출력에만 집중 할 수 있다.

또한  $\frac{∂L}{∂y} y^{2}e^{-x}$ 는 아래처럼 정리해서 쓸 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1b00UUMUUl4n5yJDi0pBZpqRPMaVhiel6' width = 550/><br>

+ 이처럼 Sigmoid 계층의 역전파는 순전파의 출력(y)만으로 계산할 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1Cjimaz7lKU0sbGYsZbP8Ury52gnn0Byd' width = 550 /><br>

+ sigmoid 계층을 파이썬으로 구현해보자.

In [ ]:
class Sigmoid:
  def __init__(self):
    self.out = None

  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out

    return dx

+ 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용한다.

## 5.6 Affine / Softmax 계층 구현하기

### 5.6.1 Affine 계층

+ 신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 곱을 사용했다.

In [ ]:
X = np.random.rand(2) # 입력
W = np.random.rand(2,3) # 가중치
B = np.random.rand(3) # 편향
Y = np.dot(X, W) + B

print(X.shape)
print(W.shape)
print(B.shape)
print(Y)

(2,)
(2, 3)
(3,)
[0.36677571 0.5352985  0.61640711]


+ 여기에서 X, W, B는 각각 형상이 (2,), (2,3), (3,)인 다차원 배열이다.
+ 뉴런의 가중치 합은 Y = np.dot(X, W) + B로 계산한다.
+ Y를 활성화 함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름이다.

<img src='http://drive.google.com/uc?export=view&id=1I5SvD_Yslv2bhnNxj0IGEHFAbf_SZsfX' width = 550 /><br>

__NOTE__  
신경망의 순전파 때 수행하는 행렬곱은 기하학에서 __어파인 변환__이라고 한다.  
여기서는 어파인 변환을 수행하는 처리를 'Affine 계층'이라는 이름으로 구현한다.

+ 앞에서 수행한 계산을 계산 그래프로 그려보자.

<img src='http://drive.google.com/uc?export=view&id=1_qubCauhlxi3Z08PO1-8AWdtx8zJ8Qic' width = 550 /><br>

+ 곱을 계산하는 노드를 'dot'라고 하면 np.dot(X, W) + B 계산은 위 그림처럼 그려진다.
+ 각 변수의 이름 위에 그 변수의 형상도 표기한다.
+ 지금까지의 계산 그래프는 노드 사이에 '스칼라 값'이 흘렀는데 이 예는 '행렬'이 흐르는 것이다.

+ 위 그림의 역전파에 대해 생각해보자.
+ 행렬을 사용한 역전파도 행렬의 원소마다 전개해보면  
스칼라값을 사용한 계산 그래프와 같은 순서로 생각할 수 있다.
+ 전개한 식은 다음과 같다.

$ \Large \frac{∂L}{∂X} = \frac{∂L}{∂Y}·W^{T}$ <br/><br/>
$ \Large \frac{∂L}{∂W} = X^{T}·\frac{∂L}{∂Y}$

+ $W^{T}$의 T는 전치행렬을 뜻한다.
+ 위 식을 바탕으로 계산 그래프의 역전파를 구해보자.

<img src='http://drive.google.com/uc?export=view&id=12DIyALPnEne5OqqWCdpeOkqd_ja9X2kz' width = 550/><br>

+ 위 계산 그래프에서 각 변수의 형상에 주의해서 보자.
+ $X$와 $\frac{∂L}{∂X}$는 같은 형상이고, $W$와 $\frac{∂L}{∂W}$도 같은 형상이다.

$\large X = (x_0, x_1, ···, x_n)$ <br/><br/>
$\large \frac{∂L}{∂X} = (\frac{∂L}{∂x_0}, \frac{∂L}{∂x_1}, ···, \frac{∂L}{∂x_2})$

+ 행렬의 형상에 주의해야 하는 이유는  
행렬의 곱에서 대응하는 차원의 원소 수를 일치시켜야 하기 때문이다.

<img src='http://drive.google.com/uc?export=view&id=1W809Wwt5RNKUYPXAD0x2ieTrYAl2xC3I' width = 550/><br>

### 5.6.2 배치용 Affine 계층

+ 지금까지 Affine 계층은 입력 데이터로 $X$ 하나만을 고려한 것이다.
+ 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해보자.
+ 배치용 Affine 계층을 게산 그래프로 그려보자.

<img src='http://drive.google.com/uc?export=view&id=16J9JgubXtdxj7Lb5VbjXQTuJiWcZZnOU' width =550/><br>

+ 기존과 다른 부분은 입력인 $X$의 형상이 (N, 2)가 된 것뿐이다.
+ 지금까지와 같이 계산 그래프의 순서를 따라 행렬 계산을 하게 된다.
+ 역전파 때는 행렬의 형상에 주의하면 $\frac{∂L}{∂X}$과 $\frac{∂L}{∂W}$은 이전과 같이 도출할 수 있다.
+ 순전파 때의 편향 덧셈은 $X·W$에 대한 편향이 각 데이터에 더해진다. 구체적인 예를 살펴보자.

In [ ]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


+ 순전파의 편향 덧셈은 각각의 데이터에 더해진다.
+ 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 한다.


In [ ]:
dY = np.array([[1,2,3],[4,5,6]])
print(dY)

dB = np.sum(dY, axis = 0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


+ Affine 구현은 다음과 같다.

In [ ]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b

    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis = 0)

    return dx

### 5.6.3 Softmax-with-Loss 계층

+ 출력층에서 사용하는 소프트맥스 함수는 입력 값을 정규화하여 출력한다.
+ 예를 들어 손글씨 숫자 인식에서의 Softmax 계층의 출력은 다음과 같다.

<img src='http://drive.google.com/uc?export=view&id=1KRtyBih-zXLQFi1-CKerMQA-ceCbzHeL' width = 550 /><br>

+ Softmax 계층은 입력 값을 정규화(출력의 합이 1이 되도록 변형)하여 출력한다.
+ 손글씨 숫자는 가짓수가 10개(10클래스 분류)이므로 Softmax 계층의 입력은 10개가 된다.

__NOTE__  
신경망에서 수행하는 작업은 __학습__과 __추론__ 두 가지이다.   
추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다.   
위 그림의 신경망은 추론할 때는 마지막 Affine 계층의 출력을 인식 결과로 이용한다.  
신경망에서 정규화하지 않은 출력 결과를 __점수__라고 한다.  
즉, 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되니 Softmax 계층은 필요없다.  
반면, 신경망을 학습할 때는 Softmax 계층이 필요하다.

+ 소프트맥스 계층을 구현할 때, 손실 함수인 교차 엔트로피 오차도 포함하여  
'Softmax-with-Loss' 계층이라는 이름으로 규현한다.
+ Softmax-with-Loss 계층의 계산 그래프를 살펴보자.

<img src='http://drive.google.com/uc?export=view&id=1oE2-MjZMsnygcOd8a54zFvqPdAhh52Vv' width = 550/><br>

+ Softmax-with-Loss 계층은 다소 복잡하므로 결과만 제시한다.
+ 위 계산 그래프는 다음과 같이 간소화할 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1FvIOonDqGwQ7de18w8CEWwQUwduc9mir' width = 550/><br>

+ 위 계산 그래프에서 소프트맥스 함수는 'Softmax' 계층으로,  
교차 엔트로피 오차는 'Cross Entropy Error' 계층으로 표기한다.
+ 3클래스 분류를 가정하고 이전 계층에서 3개의 입력을 받는다.
+ Softmax 계층은 입력 $(a_1, a_2, a_3)$를 정규화하여 $(y_1, y_2, y_3)$을 출력한다.
+ Cross Entropy Error 계층은 Softmax의 출력 $(y_1, y_2, y_3)$와 정답 레이블 $(t_1, t_2, t_3)$를 받고,  
이 데이터들로부터 손실 L을 출력한다.

+ 역전파 결과에 주목하자.
+ Softmax 계층의 역전파는 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$이다.
+ $(y_1, y_2, y_3)$는 Softmax 계층의 출력이고, $(t_1, t_2, t_3)$는 정답 레이블이므로 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$는 Softmax 계층의 출력과 정답 레이블의 차분인 것이다.
+ 신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해지는 것이다.

+ 신경망 학습의 목적은 신경망의 출력이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것이다.
+ 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달해야 한다.
+  $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$라는 결과는 Softmax 계층의 출력과 정답 레이블의 차이로,  
신경망의 현재 출력과 정답 레이블의 오차를 나타낸다.

+ 구체적인 예를 살펴보자.


정답 레이블이 (0, 1, 0)일 때 Softmax 계층이 (0.3, 0.2, 0.5)를 출력했다.

+ 정답 인덱스는 1인데 출력에서는 이 때의 확률이 0.2 이므로 신경망은 제대로 인식하지 못하고 있다.
+ 이 경우 Softmax 계층의 역전파는 (0.3, -0.8, 0.5)라는 커다란 오차를 전파한다.
+ 결과적으로 Softmax 계층의 앞 계층들은 그 큰 오차로부터 큰 깨달음을 얻게 된다.

정답 레이블이 (0, 1, 0)일 때 Softmax 계층이 (0,01, 0.99, 0)을 출력했다.

+ 이 경우 Softmax 계층의 역전파가 보내는 오차는 비교적 작은 (0.01, -0.01, 0)이다.
+ 앞 계층으로 전달된 오차가 작으므로 학습하는 정도도 작아진다.

+ Softmax-with-Loss 계층을 구현해보자.

In [ ]:
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a - c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size),t] + 1e-7)) / batch_size

In [ ]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 손실
    self.y = None # softmax의 출력
    self.t = None # 정답 레이블(원-핫 벡터)

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout = 1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size

    return dx

+ 이 구현에서 3.5.2와 4.2.4에서 구현한 함수인 softmax()와 cross_entropy_error()를 이용했다.

## 5.7 오차역전파법 구현하기

+ 지금까지 구현한 계층을 조합해서 신경망을 구축해보자.

###  5.7.1 신경망 학습의 전체 그림

+ 신경망 학습의 전체 그림을 복습할 겸 신경망 학습의 순서를 살펴보자.

__전제__ <br/>
> 신경망에는 적응 가능한 가중치와 편향이 있고,  
이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다.  
신경망 학습은 4단계로 수행한다.
  

__1단계 - 미니배치__
> 훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며,  
그 미니배치의 손실 함수 값을 줄이는 것이 목표이다.

__2단계 - 기울기 산출__
> 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.  
기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.

__3단계 - 매개변수 갱신__
> 가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

__4단계 - 반복__
> 1~3단계를 반복한다.

+ 오차역전파법이 등장하는 단계는 2단계인 '기울기 산출'이다.
+ 수치 미분을 사용하면 구현은 쉽지만 계산이 오래 걸린다.
+ 오차역전파법을 이용하면 느린 수치 미분과 달리 기울기를 효율적이고 빠르게 구할 수 있다.

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

+ 2층 신경망을 TwoLayerNet 클래스로 구현한다.
+ 이 클래스의 인스턴스 변수와 메서드를 정리한 표를 살펴보자.

|인스턴스 변수|설명|
|------|---|
|params|신경망의 매개변수를 보관하는 딕셔너리 변수|
||params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향|
||params['W2']은 2번째 층의 가중치, params['b2']은 2번째 층의 편향|
|layer|신경망의 계층을 보관하는 순서가 있는 딕셔너리 변수|
||layers['Affine1], layers['Relu1']와 같이 각 계층을 순서대로 유지|
|lastLayer|신경망의 마지막 계층|
||이 예에서는 SoftmaxWithLoss 계층|

|메서드|설명|
|------|---|
|_ _ _init_ _ _(self.input_size, hidden_size,|초기화를 수행한다.|
|output_size, weight_init_std)|인수는 순서대로 입력층의 뉴런 수, 은닉층의 뉴런 수, |
||출력층의 뉴런 수, 가중치 초기화 시 정규분포 스케일|
|predict(self, x)|예측(추론)을 수행한다.|
||인수 x는 이미지 데이터|
|loss(self, x, t)|손실 함수의 값을 구현한다.|
||인수 x는 이미지 데이터, t는 정답레이블(아래 칸의 인수들도 마찬가지)|
|accuracy(self, x, t)|정확도를 구한다.|
|numerical_gradient(self, x, t)|가중치 매개변수의 기울기를 수치 미분 방식으로 구한다.|
|gradient(self, x, t)|가중치 매개변수의 기울기를 오차역전파법으로 구한다.|

+ 클래스의 구현이 좀 길지만 앞서 구현한 클래스와 공통된 부분이 많다.
+ 계층을 사용함으로써 인식 결과를 얻는 처리 (predict( ))와  
기울기를 구하는 처리(gradient( )) 계층의 전파만으로 동작이 이루어지는 것이다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os
import numpy as np
sys.path.append("/content/drive/MyDrive/deep-learning-from-scratch-master/")
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

+ 계층 생성 부분과 역전파 부분을 집중해서 보자.
+ 신경망의 계층을 OrderedDict에 보관하는데 이는 순서가 있는 딕셔너리이다.
+ 순서가 있다는 것은 딕셔너리에 추가한 순서를 기억한다는 것이다.
+ 순전파 때는 추가한 순서대로 각 계층의 forward( ) 메서드를 호출하면 처리가 완료된다.
+ 마찬가지로 역전파 떄는 계층을 반대 순서로 호출하면 된다.
+ Affine 계층과 ReLU 계층이 각자 내부에서 순전파와 역전파를 처리하고 있으니  
계층을 올바른 순서로 연결한 다음 순서대로(혹은 역순으로) 호출해주면 끝이다.

+ 신경망의 구성 요소를 '계층'으로 구현한 덕분에 신경망을 쉽게 구축할 수 있다.
+ 각 계층 내부에 구현된 순전파와 역전파를 활용해 인식 처리와 학습에 필요한 기울기를 정확하게 구해보자.

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

+ 기울기를 구하는 방법에는 수치 미분과 오차역전파법이 있다.
+ 오차역전파법은 매개변수가 많아도 효율적으로 계산할 수 있으니 수치 미분 대신해서 사용한다.
+ 수치 미분은 오차역전파법을 정확히 구현했는지 확인하는데 필요하다.
+ 수치 미분의 결과와 오차역전파법의 결과를 비교하여  
오차역전파법을 제대로 구현했는지 검증하는 작업을 __기울기 확인__ 이라고 한다.

+ 기울기 확인을 구현해보자.

In [ ]:
# coding: utf-8
import sys, os
import numpy as np
sys.path.append("/content/drive/MyDrive/deep-learning-from-scratch-master/")
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.4223373414756884e-10
b1:2.742574832341471e-09
W2:6.174502973545773e-09
b2:1.3998274405596645e-07


+ 가장 먼저 MNIST 데이터셋을 읽는다.
+ 훈련 데이터 일부를 수치 미분으로 구한 기울기와 오차역전파법으로 구한 기울기의 오차를 확인한다.
+ 여기서는 각 가중치 매개변수의 차이의 절댓값을 구하고, 이를 평균한 값이 오차가 된다.

+ 위 결과는 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작다고 말해준다.
+ 예를 들어 1번째 층의 편향 오차는 2.6e-09이다.
+ 이는 오차역전파법으로 구한 기울기도 올바름이 드러나면서 실수 없이 구현했다고 할 수 있다.

__NOTE__
> 수치 미분과 오차역전파법의 결과 오차가 0이 되는 일은 드물다.  
이는 컴퓨터가 할 수 있는 계산의 정밀도가 유한하기 때문이다.  
정밀도의 한계 때문에 오차는 대부분 0이 되지는 않지만  
올바르게 구현했다면 0에 아주 가까운 작은 값이 된다.

### 5.7.4 오차역전파법을 사용한 학습 구현하기

+ 오차역전파법을 사용한 신경망 학습을 구현해보자.

In [ ]:
# coding: utf-8
import sys, os
sys.path.append("/content/drive/MyDrive/deep-learning-from-scratch-master/")
import numpy as np
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11808333333333333 0.1191
0.9076333333333333 0.91
0.9202666666666667 0.9229
0.93515 0.9349
0.9439333333333333 0.943
0.9498166666666666 0.9495
0.9557 0.953
0.95755 0.9535
0.9634666666666667 0.9597
0.9654666666666667 0.96
0.9676833333333333 0.9638
0.9708833333333333 0.965
0.9714333333333334 0.9651
0.9738666666666667 0.9668
0.9762166666666666 0.9682
0.9771 0.9689
0.97915 0.9703


## 5.8 정리

+ 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다.
+ 계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다.
+ 계산 그래프의 순전파는 통상의 계산을 수행한다.  
+ 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다.
+ 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다.(오차역전파법)
+ 수차 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다.(기울기 확인)